### Code from Bi-directional LSTM tutorial 
https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/


In [4]:
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional

# Traditional LSTM approach
# Given a sequence of random values between 0 and 1,
# assign an output of 0,
# sum up the sequence, and
# when a threshold is reached, assign an output of 1

def get_sequence(n_timesteps):
    X = array([random() for _ in range(n_timesteps)])
    limit = n_timesteps/4.0
    y = array([0 if x < limit else 1 for x in cumsum(X)])    
    
    X = X.reshape(1, n_timesteps, 1)
    y = y.reshape(1, n_timesteps, 1)

    return X, y


In [ ]:
n_timesteps = 10

In [21]:
# Create a model for LSTM
# Input layer: 10 timesteps with one feature each
# First hidden layer: 20 memory units
# Output layer: fully, connectected, one value per timestep
# Activation: sigmoid function to predict binary value
# Wrapper layer: TimeDistributed, receives sequence from hidden layer
# Loss function: binary_crossentropy (binary log loss)
# Optimization: ADAM, assign weights
# Metric: accuracy reported for each epoch (pass as a list)
model = Sequential()
model.add(LSTM(20, input_shape=(n_timesteps, 1), return_sequences=True))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])


In [22]:
for epoch in range(1000):
    X, y = get_sequence(n_timesteps)
    model.fit(X, y, epochs=1, batch_size=1, verbose=2)


Epoch 1/1
0s - loss: 0.7014 - acc: 0.3000
Epoch 1/1
0s - loss: 0.6909 - acc: 0.8000
Epoch 1/1
0s - loss: 0.7185 - acc: 0.2000
Epoch 1/1
0s - loss: 0.7072 - acc: 0.4000
Epoch 1/1
0s - loss: 0.7019 - acc: 0.3000
Epoch 1/1
0s - loss: 0.6926 - acc: 0.4000
Epoch 1/1
0s - loss: 0.6880 - acc: 0.7000
Epoch 1/1
0s - loss: 0.6896 - acc: 0.7000
Epoch 1/1
0s - loss: 0.6950 - acc: 0.3000
Epoch 1/1
0s - loss: 0.6895 - acc: 0.6000
Epoch 1/1
0s - loss: 0.6893 - acc: 0.6000
Epoch 1/1
0s - loss: 0.6823 - acc: 0.8000
Epoch 1/1
0s - loss: 0.6827 - acc: 0.9000
Epoch 1/1
0s - loss: 0.6885 - acc: 0.7000
Epoch 1/1
0s - loss: 0.6795 - acc: 0.8000
Epoch 1/1
0s - loss: 0.6770 - acc: 0.7000
Epoch 1/1
0s - loss: 0.6688 - acc: 0.9000
Epoch 1/1
0s - loss: 0.6874 - acc: 0.5000
Epoch 1/1
0s - loss: 0.6664 - acc: 0.8000
Epoch 1/1
0s - loss: 0.6721 - acc: 0.7000
Epoch 1/1
0s - loss: 0.6798 - acc: 0.6000
Epoch 1/1
0s - loss: 0.6766 - acc: 0.5000
Epoch 1/1
0s - loss: 0.7017 - acc: 0.4000
Epoch 1/1
0s - loss: 0.6572 - acc:

In [25]:
X, y = get_sequence(n_timesteps)
yhat = model.predict_classes(X, verbose=0)

for i in range(n_timesteps):
    print('Expected:', y[0, i], 'Predicted', yhat[0, i])

Expected: [0] Predicted [0]
Expected: [0] Predicted [0]
Expected: [0] Predicted [0]
Expected: [0] Predicted [0]
Expected: [1] Predicted [0]
Expected: [1] Predicted [1]
Expected: [1] Predicted [1]
Expected: [1] Predicted [1]
Expected: [1] Predicted [1]
Expected: [1] Predicted [1]
